In [1]:
import simplejson as json
with open('../../external_data/google_place/total_place.json','r') as f_in:
    data = json.loads(f_in.readline())

# Google place - The open-access ultimate external data

In [4]:
data[1]

{'html_attributions': [],
 'next_page_token': 'CrQCJgEAALogR0B5thMR7GFGZhvtySeyfabEmI3umRFoUks4kH0l5aioxFwVxQUO5wQqbUjJqTtokD3SZOtGTQ43zq-50j12FtfvGlJ_dQ_o-k_J2YtJhMKwtFqfa1iYA6BH1Mi60GdPr4zIMVt-o5jIMTFbTX0s_qUTWk0HlE0yGw3bL8EhAsSVPRzLl-OkiOrCWg31WB6Lgd9qplsHSPrg8AsPRbFYHF5lXVlE54la3ACwOnCh-j6OJxf0Xd5oVQYk1MnYLH3nzHLzUe2xCB9p3aR30nuSLXkJod56vfA9AKBdA8k-LpAGuhmoaSbnirSz9Oq9RiCGBQN9VvbkJm3HmBeyKra8hvVqswsbznQfnb0MgHa1B8RV4rRUpAfe-1iSLlnVbe20jBuApsAbLPTBAxhfegMSEL_CYeP1-5D8AOXBPcE9ud0aFDAtVET-cfMwC0YLHXMnXX4UsY6-',
 'results': [{'geometry': {'location': {'lat': 34.6703624,
     'lng': -86.0185903}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png',
   'id': '6bb31d338f225a408c803f7f48cf2cb2f8639d8b',
   'name': 'Brownwood Elementary School',
   'place_id': 'ChIJu9ZGLNHuYYgRFKDDc9BTZ_4',
   'reference': 'CnRvAAAASiXQ7y7ux3F-xKTnDKRT8nZ6vrERq4lworoKBwAYo9SnTtk6XiJIN2mcdPezD10spamk0sagCIzK1urA1FPfUv9sSK4ZyT5LmH3Cz2iml40Z5FG7bTGF1jMrcZQyueoWPphvwYFGoJJuUT29zhf4IBIQ

In [11]:
def clean_type(type_lst):
    for bored_keyword in [ 'point_of_interest', 'establishment']:
        if bored_keyword in type_lst:
            type_lst.remove(bored_keyword)
    return type_lst
def extract_places(loc):
    place_lst = []
    for place in loc['results']:
        dic = {}
        dic['lat'] = place['geometry']['location']['lat']
        dic['lat'] = place['geometry']['location']['lng']
        dic['name'] = place['name']
        dic['types'] = clean_type(place['types'])
        place_lst.append(dic)
    return place_lst

## Cleaning the data

In [12]:
# this is a heavy computation cleaning step
cleaned_data = [extract_places(record) for record in data]

In [13]:
cleaned_data[1]

[{'lat': -86.0185903,
  'name': 'Brownwood Elementary School',
  'types': ['school']},
 {'lat': -86.0027441, 'name': 'Scottsboro High School', 'types': ['school']},
 {'lat': -86.0390437,
  'name': 'Collins Elementary School',
  'types': ['school']},
 {'lat': -86.03275719999999,
  'name': 'Caldwell Elementary School',
  'types': ['school']},
 {'lat': -85.9726555,
  'name': 'Hollywood Junior High School',
  'types': ['school']},
 {'lat': -86.01379399999999,
  'name': 'Ruby Tuesday',
  'types': ['bar', 'restaurant', 'food']},
 {'lat': -86.0226674,
  'name': 'MJ FUEL & SPIRIT',
  'types': ['liquor_store',
   'convenience_store',
   'food',
   'store',
   'gas_station']},
 {'lat': -86.0027599, 'name': 'Scottsboro High School', 'types': ['school']},
 {'lat': -85.8440914, 'name': 'Pisgah High School', 'types': ['school']},
 {'lat': -85.94481259999999,
  'name': 'Epruett Center of Technology',
  'types': ['school']},
 {'lat': -86.0332042, 'name': 'Tiny Tot Day Care Center', 'types': ['school']